# Preprocessing phase

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('../Data/export.csv')

/var/folders/pl/lk042g854hb52mqtvl48gvyc0000gn/T/ipykernel_96904/3925814633.py:1: DtypeWarning: Columns (12,22) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('../Data/export.csv')


In [3]:
columnsName = ['db_sc', 'id_sc', 'pv', 'data', 'cassa', 'cassiere', 'num_scontrino', 'ora', 'tessera', 't_flag', 'num_riga',
                'r_reparto_cdaplus', 'r_ean', 'r_qta_pezzi', 'r_peso', 'r_importo_lordo', 'r_imponibile', 'r_iva', 'r_sconto',
                'r_sconto_fide', 'r_sconto_rip', 'r_tipo_riga', 'cod_prod', 'descr_prod', 'cat_mer', 'cod_forn', 'descr_forn',
                'liv1', 'descr_liv1', 'liv2', 'descr_liv2', 'liv3', 'descr_liv3', 'liv4', 'descr_liv4', 'liv5', 'descr_liv5',
                'liv6', 'descr_liv6', 'tipologia', 'descr_tipologia', 'cod_rep', 'descr_rep']

dataset.columns = columnsName

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
dataset

,db_sc,id_sc,pv,data,cassa,cassiere,num_scontrino,ora,tessera,t_flag,num_riga,r_reparto_cdaplus,r_ean,r_qta_pezzi,r_peso,r_importo_lordo,r_imponibile,r_iva,r_sconto,r_sconto_fide,r_sconto_rip,r_tipo_riga,cod_prod,descr_prod,cat_mer,cod_forn,descr_forn,liv1,descr_liv1,liv2,descr_liv2,liv3,descr_liv3,liv4,descr_liv4,liv5,descr_liv5,liv6,descr_liv6,tipologia,descr_tipologia,cod_rep,descr_rep
0,sysevo,52597225,4653,2023-03-25,2,2,248,21:08,NaN,V,13,1,5449000005090,1,0.0,1.29,1.057377,22,0.50,0,0.00,NORMALE,1000931,"1000931 FANTA PET LT.1,5 (P)",2721,407,00407 SIBEG SRL,02,BEVANDE,0207,BEVANDE GASSATE,020702,ARANCIATA,02070201,ARANCIATA ZUCCHERO,NaN,NaN,NaN,NaN,NO,Prodotti Normali,001,ALIMENTARE
1,sysevo,52597232,4542,2023-03-25,1,1,581,21:00,2.484252e+11,V,1,1,8006763007333,1,0.0,0.78,0.750000,4,0.47,0,0.10,NORMALE,1004624,1004624 ZAPPALA LATTE UHT P/S LT.1,1I23,868,00868 ZAPPALA' SPA *,01,DROGHERIA ALIMENTARE,0118,LATTICINI UHT E ASSIMILABILI,011802,LATTE UHT,01180203,PARZIALMENTE SCREMATO,NaN,NaN,NaN,NaN,NO,Prodotti Normali,001,ALIMENTARE
2,sysevo,52597232,4542,2023-03-25,1,1,581,21:00,2.484252e+11,V,17,5,8003980090056,1,0.0,2.64,2.163934,22,0.35,0,0.35,NORMALE,1090781,1090781 CUKI TEGLIA EXTRA S97 8POR.X2,5442,4474,04474 CUKI COFRESCO S.R.L.,05,CURA CASA,0504,USA E GETTA,050404,SACCHETTI/VASCHETTE ALIMENT,05040402,VASCHETTE,NaN,NaN,NaN,NaN,NO,Prodotti Normali,005,CURA CASA
3,sysevo,52597232,4542,2023-03-25,1,1,581,21:00,2.484252e+11,V,19,3,8001120932228,1,0.0,1.41,1.281818,10,0.18,0,0.18,NORMALE,3000305,3000305 COOP SFOGLIA RETT.S/O.PAL230G,3823,5447,05447 COOP PAM (CERELIA ITALIA S.R.L.),03,FRESCO,0308,ALTRI PRODOTTI FRESCHI,030802,BASI,03080203,PASTA SFOGLIA,NaN,NaN,NaN,NaN,PM,Prodotti a Marchio Coop,003,BANCO LIBERO SERVIZIO
4,sysevo,52597232,4542,2023-03-25,1,1,581,21:00,2.484252e+11,V,21,3,8001120932228,1,0.0,1.41,1.281818,10,0.18,0,0.18,NORMALE,3000305,3000305 COOP SFOGLIA RETT.S/O.PAL230G,3823,5447,05447 COOP PAM (CERELIA ITALIA S.R.L.),03,FRESCO,0308,ALTRI PRODOTTI FRESCHI,030802,BASI,03080203,PASTA SFOGLIA,NaN,NaN,NaN,NaN,PM,Prodotti a Marchio Coop,003,BANCO LIBERO SERVIZIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9682403,sysevo,62228224,4542,2023-09-30,5,5,375,21:10,NaN,V,7,1,8001860185069,1,0.0,1.39,1.263636,10,0.00,0,0.00,NORMALE,1037699,1037699 SCOTTI SNACK VENERE BBQ GR50 P,1321,432,00432 RISO SCOTTI SPA @,01,DROGHERIA ALIMENTARE,0103,FUORI PASTO SALATI,010302,ESTRUSI/MAIS/FORMAGGIO,01030201,MAIS/FORMAGGIO,NaN,NaN,NaN,NaN,NO,Prodotti Normali,001,ALIMENTARE
9682404,sysevo,62228224,4542,2023-09-30,5,5,375,21:10,NaN,V,9,1,8025579220088,1,0.0,0.99,0.900000,10,0.00,0,0.00,NORMALE,1061126,1061126 PREZIOSI MAIS CORNET.FORM.100G,1321,1577,01577 PREZIOSI FOOD SPA,01,DROGHERIA ALIMENTARE,0103,FUORI PASTO SALATI,010302,ESTRUSI/MAIS/FORMAGGIO,01030201,MAIS/FORMAGGIO,NaN,NaN,NaN,NaN,NO,Prodotti Normali,001,ALIMENTARE
9682405,sysevo,62228225,4542,2023-09-30,3,3,303,21:12,2.484252e+11,V,1,1,8001120720948,1,0.0,0.69,0.627273,10,0.00,0,0.00,NORMALE,3501904,3501904 COOP ACETO BIANCO ALCOL! LT.1,1511,6259,06259 COOP PAM (ACETIFICI ITALIANI MODENA SRL),01,DROGHERIA ALIMENTARE,0105,"OLIO,ACETO E SUCCO DI LIMONE",010501,ACETO,01050101,STANDARD,NaN,NaN,NaN,NaN,PM,Prodotti a Marchio Coop,001,ALIMENTARE
9682406,sysevo,62228225,4542,2023-09-30,3,3,303,21:12,2.484252e+11,V,3,1,8000500391228,1,0.0,3.39,3.081818,10,0.00,0,0.00,NORMALE,1138155,1138155 KINDER KINDERINI T20X10 G250,1142,4019,04019 FERRERO COMMERCIALE ITALIA S.R.L.(SDDU),01,DROGHERIA ALIMENTARE,0101,PRODOTTI FORNO E CEREALI,010104,BISCOTTI,01010402,FROLLINI ARRICCHITI,NaN,NaN,NaN,NaN,NO,Prodotti Normali,001,ALIMENTARE


In [6]:
dataset['cod_prod'] = dataset['cod_prod'].astype(str)
dataset['cod_prod'] = dataset['cod_prod'].apply(lambda x: x.split('.')[0] if '.0' in x else x)


In [7]:
dataset = dataset.dropna(subset=['cod_prod'])
dataset = dataset[dataset['r_tipo_riga'] != 'ANNULLO']
dataset = dataset[dataset['r_importo_lordo'] != 0]
dataset = dataset[dataset['cat_mer'] != 'NNNNN']
dataset = dataset[dataset['cod_prod'] != '1090011'] # shopping bag

For implement a Recommendation system, we must know who is the client.

The only feature that return this information is 'tessera' but the fidelity customers are much less.

In [8]:
dataset = dataset[dataset['tessera'].notna()]

In [9]:
dataset['data'] = pd.to_datetime(dataset['data'])
dataset['ora'] = pd.to_datetime(dataset['ora'], format='%H:%M').dt.time

In [10]:
df = dataset[['id_sc', 'pv', 'data', 'ora', 'tessera', 'r_qta_pezzi', 'r_peso', 'r_importo_lordo', 'r_sconto', 'r_sconto_rip', 'cod_prod', 'descr_prod',
                'cat_mer', 'cod_forn', 'liv1', 'descr_liv1', 'liv2', 'descr_liv2', 'liv3', 'descr_liv3', 'liv4', 'descr_liv4']]

In [11]:
df

,id_sc,pv,data,ora,tessera,r_qta_pezzi,r_peso,r_importo_lordo,r_sconto,r_sconto_rip,cod_prod,descr_prod,cat_mer,cod_forn,liv1,descr_liv1,liv2,descr_liv2,liv3,descr_liv3,liv4,descr_liv4
1,52597232,4542,2023-03-25,21:00:00,2.484252e+11,1,0.000000,0.78,0.47,0.10,1004624,1004624 ZAPPALA LATTE UHT P/S LT.1,1I23,868,01,DROGHERIA ALIMENTARE,0118,LATTICINI UHT E ASSIMILABILI,011802,LATTE UHT,01180203,PARZIALMENTE SCREMATO
2,52597232,4542,2023-03-25,21:00:00,2.484252e+11,1,0.000000,2.64,0.35,0.35,1090781,1090781 CUKI TEGLIA EXTRA S97 8POR.X2,5442,4474,05,CURA CASA,0504,USA E GETTA,050404,SACCHETTI/VASCHETTE ALIMENT,05040402,VASCHETTE
3,52597232,4542,2023-03-25,21:00:00,2.484252e+11,1,0.000000,1.41,0.18,0.18,3000305,3000305 COOP SFOGLIA RETT.S/O.PAL230G,3823,5447,03,FRESCO,0308,ALTRI PRODOTTI FRESCHI,030802,BASI,03080203,PASTA SFOGLIA
4,52597232,4542,2023-03-25,21:00:00,2.484252e+11,1,0.000000,1.41,0.18,0.18,3000305,3000305 COOP SFOGLIA RETT.S/O.PAL230G,3823,5447,03,FRESCO,0308,ALTRI PRODOTTI FRESCHI,030802,BASI,03080203,PASTA SFOGLIA
5,52597232,4542,2023-03-25,21:00:00,2.484252e+11,1,0.000000,0.44,1.55,0.06,3000397,3000397 *COOP P.U.GRAM.COOP N/ANT G500,1F61,5543,01,DROGHERIA ALIMENTARE,0115,PASTA,011506,PASTA ALL'UOVO,01150601,BRODI/MINESTRINE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9682386,62224004,4659,2023-09-30,21:26:00,2.484252e+11,1,0.226263,2.24,0.00,0.00,1037398,1037398 COATI PROSCIUTTO COTTO ALLEGRO,3642,425,03,FRESCO,0306,SALUMI,030604,SALUMI BANCO TAGLIO,03060402,PROSCIUTTO COTTO
9682387,62224004,4659,2023-09-30,21:26:00,2.484252e+11,1,0.083838,0.83,0.00,0.00,1055261,1055261 AMADORI ARROSTO TACCHINO 1/2,3649,254,03,FRESCO,0306,SALUMI,030604,SALUMI BANCO TAGLIO,03060409,ARROSTI
9682405,62228225,4542,2023-09-30,21:12:00,2.484252e+11,1,0.000000,0.69,0.00,0.00,3501904,3501904 COOP ACETO BIANCO ALCOL! LT.1,1511,6259,01,DROGHERIA ALIMENTARE,0105,"OLIO,ACETO E SUCCO DI LIMONE",010501,ACETO,01050101,STANDARD
9682406,62228225,4542,2023-09-30,21:12:00,2.484252e+11,1,0.000000,3.39,0.00,0.00,1138155,1138155 KINDER KINDERINI T20X10 G250,1142,4019,01,DROGHERIA ALIMENTARE,0101,PRODOTTI FORNO E CEREALI,010104,BISCOTTI,01010402,FROLLINI ARRICCHITI


In [12]:
df.to_csv('../Data/export_clean.csv', index=False)